# Data Prepocessing

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm

## 1. Data Load

In [ ]:
# 20대 국회의원 법안발의안
# [출처] DACON 국회 입법활동 빅데이터 시각화 경진대회: https://www.dacon.io/competitions/official/235679/data/

df_suggest = pd.read_csv('./suggest.csv')
df_suggest = df_suggest[df_suggest.AGE == 20] 

In [ ]:
# 20대 국회의원 명당
# [출처] 열린국회정보 정보공개포털 : https://open.assembly.go.kr/portal/assm/search/memberHistSchPage.do

df_ppl = pd.read_excel('의원검색 (1).xlsx')

In [ ]:
df_suggest.head(3)

,BILL_ID,BILL_NO,BILL_NAME,COMMITTEE,PROPOSE_DT,PROC_RESULT,AGE,DETAIL_LINK,PROPOSER,MEMBER_LIST,RST_PROPOSER,PUBL_PROPOSER
0,PRC_A2T0Z0E9U1I6P1G3P3X9I3B7H7D9C1,2104402,독점규제 및 공정거래에 관한 법률 일부개정법률안,NaN,2020-10-05,NaN,21,http://likms.assembly.go.kr/bill/billDetail.do...,박홍근의원등12인,http://likms.assembly.go.kr/bill/coactorListPo...,박홍근,"김원이,남인순,서영석,송옥주,용혜인,윤미향,이용빈,장경태,정태호,주철현,최종윤"
1,PRC_I2M0O1G0U0P5E1U8T4B6Z0R9M8G5X0,2104401,지방교육재정교부금법 일부개정법률안,NaN,2020-10-05,NaN,21,http://likms.assembly.go.kr/bill/billDetail.do...,서영교의원 등 10인,http://likms.assembly.go.kr/bill/coactorListPo...,서영교,"김병기,서삼석,신정훈,오영환,윤재갑,이개호,이성만,전혜숙,최형두"
2,PRC_C2Q0L1M0V0B5G1O8H1N6Z5V0W5W7A7,2104400,농업기계화 촉진법 일부개정법률안,NaN,2020-10-05,NaN,21,http://likms.assembly.go.kr/bill/billDetail.do...,윤재갑의원 등 11인,http://likms.assembly.go.kr/bill/coactorListPo...,윤재갑,"강병원,김수흥,서영교,위성곤,이개호,이재정,이정문,전용기,조경태,홍문표"


In [ ]:
df_suggest.columns

Index(['BILL_ID', 'BILL_NO', 'BILL_NAME', 'COMMITTEE', 'PROPOSE_DT',
       'PROC_RESULT', 'AGE', 'DETAIL_LINK', 'PROPOSER', 'MEMBER_LIST',
       'RST_PROPOSER', 'PUBL_PROPOSER'],
      dtype='object')

In [ ]:
df_ppl.head(3)

,번호,대수,의원명,정당,소속위원회,지역,성별,당선횟수,당선방법
0,1,제20대,강길부,무소속,NaN,울산 울주군,남,4선,지역구
1,2,제20대,강병원,더불어민주당,NaN,서울 은평구을,남,재선,지역구
2,3,제20대,강석진,새누리당,NaN,경남 산청군함양군거창군합천군,남,초선,지역구


In [ ]:
df_ppl.columns

Index(['번호', '대수', '의원명', '정당', '소속위원회', '지역', '성별', '당선횟수', '당선방법'], dtype='object')

## 2. 전처리

### 대수 숫자만 남기기

In [ ]:
df_ppl['대수'] = df_ppl['대수'].str.slice(start=1, stop=3)

In [ ]:
df_ppl.head(2)

,번호,대수,의원명,정당,소속위원회,지역,성별,당선횟수,당선방법
0,1,20,강길부,무소속,NaN,울산 울주군,남,4선,지역구
1,2,20,강병원,더불어민주당,NaN,서울 은평구을,남,재선,지역구


### 법률안에서 Keyword 뽑기

In [ ]:
# https://konlpy.org/ko/v0.5.2/install/
# !pip install konlpy

from konlpy.tag import Okt  
okt=Okt()

In [ ]:
# stopwords : 미리 tokenize해본 후 중복되는 상위 keyword 추출

stopwords = ['법률','안','개정','일부','법','관','및','등','관리','지원',
 '특별법','기본','특례','진흥','보호','산업','법안','사업','안전',
 '세법','촉진','운영','보험','국가','이용','특별','사법','설치','위'] 


word_list = []
for txt in tqdm(df_suggest.BILL_NAME):
    temp = okt.nouns(txt)
    temp = [word for word in temp if not word in stopwords if len(word) != 1] # 1글자 제거
    word_list.append(temp)

In [ ]:
from collections import Counter

# 상위 키워드 확인
keyword_total = [word for words in word_list for word in words]
Counter(keyword_total).most_common()

[('국민', 1064),
 ('공직선거법', 1047),
 ('거래', 1010),
 ('고용', 966),
 ('개발', 857),
 ('설립', 857),
 ('정법', 841),
 ('한국', 825),
 ('처벌', 822),
 ('지방', 813),
 ('국회법', 790),
 ('건강', 789),
 ('지역', 781),
 ('특례법', 762),
 ('보건', 753),
 ('복지', 734),
 ('도시', 702),
 ('환경', 693),
 ('도로교통법', 692),
 ('범죄', 690),
 ('보상', 639),
 ('육성', 628),
 ('건설', 615),
 ('발전', 581),
 ('소득', 570),
 ('교육', 570),
 ('금융', 553),
 ('공무원', 538),
 ('규제', 532),
 ('예방', 525),
 ('중소기업', 503),
 ('보장', 500),
 ('지방세', 494),
 ('촉진법', 482),
 ('정화', 482),
 ('방지', 477),
 ('의료법', 461),
 ('사회', 460),
 ('근로기준법', 457),
 ('유공', 453),
 ('장애인', 447),
 ('청소년', 447),
 ('시설', 443),
 ('의료', 426),
 ('공공기관', 426),
 ('조치', 425),
 ('학교', 421),
 ('교육법', 419),
 ('조성', 416),
 ('문화', 414),
 ('피해자', 411),
 ('기금', 407),
 ('방송', 394),
 ('협력', 388),
 ('구역', 384),
 ('군인', 380),
 ('기술', 380),
 ('대책', 374),
 ('정보보호', 373),
 ('중등교육', 373),
 ('성폭력', 371),
 ('연금', 369),
 ('노인', 364),
 ('형법', 357),
 ('영유아보육법', 356),
 ('정보통신망', 343),
 ('교통', 341),
 ('자동차', 341),
 ('택법', 3

In [ ]:
df_suggest['keyword'] = [",".join(words) for words in word_list]

<ipython-input-107-021d9d83cb3c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_suggest['keyword'] = [",".join(words) for words in word_list]


In [ ]:
df_suggest = df_suggest[df_suggest.AGE == 20].dropna().sample(n=3500, random_state=3) # random하게 3500 case만 추출
df_suggest = df_suggest.dropna(axis=0, subset=['PUBL_PROPOSER','keyword'])

### node,edge feature에 한글이 들어가면 저장이 안되는 문제 발생 -> index 부여

In [ ]:
party_list = df_ppl['정당'].unique()

party2idx = { party : i for i,party in enumerate(party_list)}
party2idx

{'무소속': 0, '더불어민주당': 1, '새누리당': 2, '국민의당': 3, '자유한국당': 4, '정의당': 5, '바른미래당': 6}

In [ ]:
df_ppl['party_index'] = df_ppl['정당'].apply(lambda x: party2idx[x])
df_ppl

,번호,대수,의원명,정당,소속위원회,지역,성별,당선횟수,당선방법,party_index
0,1,20,강길부,무소속,NaN,울산 울주군,남,4선,지역구,0
1,2,20,강병원,더불어민주당,NaN,서울 은평구을,남,재선,지역구,1
2,3,20,강석진,새누리당,NaN,경남 산청군함양군거창군합천군,남,초선,지역구,2
3,4,20,강석호,새누리당,NaN,경북 영양군영덕군봉화군울진군,남,3선,지역구,2
4,5,20,강창일,더불어민주당,NaN,제주 제주시갑,남,4선,지역구,1
...,...,...,...,...,...,...,...,...,...,...
315,316,20,홍일표,새누리당,NaN,인천 남구갑,남,3선,지역구,2
316,317,20,홍철호,새누리당,NaN,경기 김포시을,남,재선,지역구,2
317,318,20,황영철,새누리당,NaN,강원 홍천군철원군화천군양구군인제군,남,3선,지역구,2
318,319,20,황주홍,국민의당,NaN,전남 고흥군보성군장흥군강진군,남,재선,지역구,3


In [ ]:
# node,edge feature에 한글이 들어가면 저장이 안되는 문제 발생 -> index 부여
COMMITTEE_list = df_suggest.COMMITTEE.unique()

COMMITTEE2idx = { COMMITTEE : i for i,COMMITTEE in enumerate(COMMITTEE_list)}
COMMITTEE2idx

{'법제사법위원회': 0,
 '농림축산식품해양수산위원회': 1,
 '국토교통위원회': 2,
 '보건복지위원회': 3,
 '교육문화체육관광위원회': 4,
 '행정안전위원회': 5,
 '환경노동위원회': 6,
 '문화체육관광위원회': 7,
 '외교통일위원회': 8,
 '교육위원회': 9,
 '정무위원회': 10,
 '산업통상자원중소벤처기업위원회': 11,
 '산업통상자원위원회': 12,
 '기획재정위원회': 13,
 '국방위원회': 14,
 '과학기술정보방송통신위원회': 15,
 '국회운영위원회': 16,
 '안전행정위원회': 17,
 '여성가족위원회': 18,
 '미래창조과학방송통신위원회': 19,
 '정치개혁 특별위원회': 20,
 '정보위원회': 21,
 '헌법개정 및 정치개혁 특별위원회': 22}

In [ ]:
df_suggest['COMMITTEE_index'] = df_suggest['COMMITTEE'].apply(lambda x: COMMITTEE2idx[x])
df_suggest

,BILL_ID,BILL_NO,BILL_NAME,COMMITTEE,PROPOSE_DT,PROC_RESULT,AGE,DETAIL_LINK,PROPOSER,MEMBER_LIST,RST_PROPOSER,PUBL_PROPOSER,keyword,COMMITTEE_index
0,PRC_B1A6Z0N7H2Q6C1C0D3N0Q5K6S5U4Y2,2001183,집단소송법안,법제사법위원회,2016-07-26,임기만료폐기,20,http://likms.assembly.go.kr/bill/billDetail.do...,박영선의원 등 46인,http://likms.assembly.go.kr/bill/coactorListPo...,박영선,"강훈식,기동민,김경수,김두관,김병욱,김상희,김성수,김영주,김영춘,김종민,김종인,김현...",집단소송,0
1,PRC_A1B9W1O1O0B8Z1N8C4Z1M4D0E5S3J4,2023701,수산업법 일부개정법률안,농림축산식품해양수산위원회,2019-11-08,대안반영폐기,20,http://likms.assembly.go.kr/bill/billDetail.do...,박주현의원 등 10인,http://likms.assembly.go.kr/bill/coactorListPo...,박주현,"강석진,경대수,김성찬,손금주,손혜원,오영훈,윤준호,이양수,황주홍",수산업,1
2,PRC_G1C6M0U8R1O1O0A9C4G1S1I3M7J8J4,2001532,자동차관리법 일부개정법률안,국토교통위원회,2016-08-11,임기만료폐기,20,http://likms.assembly.go.kr/bill/billDetail.do...,임종성의원 등 11인,http://likms.assembly.go.kr/bill/coactorListPo...,임종성,"강병원,김경협,김병욱,김철민,민홍철,소병훈,안호영,윤후덕,전현희,최경환",자동차관리법,2
3,PRC_E1H6R1P1W2U9L1N6D5U7M4W8D6K0G1,2003948,집합건물의 소유 및 관리에 관한 법률 일부개정법률안,법제사법위원회,2016-11-29,임기만료폐기,20,http://likms.assembly.go.kr/bill/billDetail.do...,전현희의원 등 10인,http://likms.assembly.go.kr/bill/coactorListPo...,전현희,"기동민,김영춘,노웅래,박선숙,손혜원,윤호중,윤후덕,임종성,정성호","집합,건물,소유",0
4,PRC_P1B7H0I8V3O0D1H4O2O9F2A1I4T6U3,2008863,장기공공임대주택 입주자 삶의 질 향상 지원법 일부개정법률안,국토교통위원회,2017-08-30,수정가결,20,http://likms.assembly.go.kr/bill/billDetail.do...,전현희의원 등 10인,http://likms.assembly.go.kr/bill/coactorListPo...,전현희,"김정우,신창현,안규백,안호영,윤관석,이원욱,임종성,주승용,한정애","장기,임대주택,주자,향상",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,PRC_C1C6R1U2Q2M6Y1V4N5L9G3K1R7V9D9,2004610,고등교육법 일부개정법률안,교육위원회,2016-12-26,임기만료폐기,20,http://likms.assembly.go.kr/bill/billDetail.do...,서영교의원 등 10인,http://likms.assembly.go.kr/bill/coactorListPo...,서영교,"강창일,김병기,김종대,남인순,박재호,안규백,위성곤,전혜숙,황희","고등,교육법",9
3496,PRC_E1Z8A1B2T0B6B2Q0U0M9L4X7A3Y0U7,2017278,광산피해의 방지 및 복구에 관한 법률 일부개정법률안,산업통상자원중소벤처기업위원회,2018-12-06,임기만료폐기,20,http://likms.assembly.go.kr/bill/billDetail.do...,황주홍의원 등 10인,http://likms.assembly.go.kr/bill/coactorListPo...,황주홍,"박주현,이언주,이용주,이종걸,이찬열,장정숙,정동영,정인화,최도자","광산,피해,방지,복구",11
3497,PRC_V1N7G1D2E2P9Q1I4A5G4E4P4Z0U5W1,2011138,소금산업 진흥법 일부개정법률안,농림축산식품해양수산위원회,2017-12-29,대안반영폐기,20,http://likms.assembly.go.kr/bill/billDetail.do...,전혜숙의원 등 12인,http://likms.assembly.go.kr/bill/coactorListPo...,전혜숙,"기동민,김상희,김정우,노웅래,신창현,양승조,이찬열,인재근,장정숙,전해철,정춘숙",소금,1
3498,PRC_R1D9C0E9X0U3M1T1H3Y3A5N6P8C1L8,2022311,도로교통법 일부개정법률안,행정안전위원회,2019-09-03,임기만료폐기,20,http://likms.assembly.go.kr/bill/billDetail.do...,백승주의원 등 10인,http://likms.assembly.go.kr/bill/coactorListPo...,백승주,"김무성,박덕흠,박명재,유민봉,장석춘,최교일,최연혜,홍철호,황영철",도로교통법,5


In [ ]:
BILL_list = df_suggest.BILL_NAME.unique()

BILL2idx = { BILL : i for i,BILL in enumerate(BILL_list)}
BILL2idx

{'집단소송법안': 0,
 '수산업법 일부개정법률안': 1,
 '자동차관리법 일부개정법률안': 2,
 '집합건물의 소유 및 관리에 관한 법률 일부개정법률안': 3,
 '장기공공임대주택 입주자 삶의 질 향상 지원법 일부개정법률안': 4,
 '식품위생법 일부개정법률안': 5,
 '정신건강증진 및 정신질환자 복지서비스 지원에 관한 법률 일부개정법률안': 6,
 '문화재보호법 일부개정법률안': 7,
 '지방자치법 일부개정법률안': 8,
 '남녀고용평등과 일·가정 양립 지원에 관한 법률 일부개정법률안': 9,
 '예술인 복지법 일부개정법률안': 10,
 '고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 일부개정법률안': 11,
 '남북교류협력에 관한 법률 일부개정법률안': 12,
 '초·중등교육법 일부개정법률안': 13,
 '도시 및 주거환경정비법 일부개정법률안': 14,
 '최저임금법 일부개정법률안': 15,
 '미용업의 진흥 및 관리에 관한 법률안': 16,
 '상호저축은행법 일부개정법률안': 17,
 '환경범죄 등의 단속 및 가중처벌에 관한 법률 일부개정법률안': 18,
 '대기환경보전법 일부개정법률안': 19,
 '도로교통법 일부개정법률안': 20,
 '여신전문금융업법 일부개정법률안': 21,
 '지역신용보증재단법 일부개정법률안': 22,
 '법률구조법 일부개정법률안': 23,
 '벤처기업육성에 관한 특별조치법 일부개정법률안': 24,
 '지방세특례제한법 일부개정법률안': 25,
 '지열발전사업으로 촉발된  포항지진의 진상조사 및 피해구제 등을 위한 특별법안': 26,
 '응급의료에 관한 법률 일부개정법률안': 27,
 '장애인·노인 등을 위한 보조기기 지원 및 활용촉진에 관한 법률 일부개정법률안': 28,
 '의료법 일부개정법률안': 29,
 '디자인보호법 일부개정법률안': 30,
 '검찰청법 일부개정법률안': 31,
 '대부업 등의 등록 및 금융이용자 보호에 관한 법률 일부개정법률안': 32,
 '주거급여법 일부개정법률안': 33,
 '장애인복지법 

In [ ]:
df_suggest['BILL_index'] = df_suggest['BILL_NAME'].apply(lambda x: BILL2idx[x])
df_suggest

,BILL_ID,BILL_NO,BILL_NAME,COMMITTEE,PROPOSE_DT,PROC_RESULT,AGE,DETAIL_LINK,PROPOSER,MEMBER_LIST,RST_PROPOSER,PUBL_PROPOSER,keyword,COMMITTEE_index,BILL_index
0,PRC_B1A6Z0N7H2Q6C1C0D3N0Q5K6S5U4Y2,2001183,집단소송법안,법제사법위원회,2016-07-26,임기만료폐기,20,http://likms.assembly.go.kr/bill/billDetail.do...,박영선의원 등 46인,http://likms.assembly.go.kr/bill/coactorListPo...,박영선,"강훈식,기동민,김경수,김두관,김병욱,김상희,김성수,김영주,김영춘,김종민,김종인,김현...",집단소송,0,0
1,PRC_A1B9W1O1O0B8Z1N8C4Z1M4D0E5S3J4,2023701,수산업법 일부개정법률안,농림축산식품해양수산위원회,2019-11-08,대안반영폐기,20,http://likms.assembly.go.kr/bill/billDetail.do...,박주현의원 등 10인,http://likms.assembly.go.kr/bill/coactorListPo...,박주현,"강석진,경대수,김성찬,손금주,손혜원,오영훈,윤준호,이양수,황주홍",수산업,1,1
2,PRC_G1C6M0U8R1O1O0A9C4G1S1I3M7J8J4,2001532,자동차관리법 일부개정법률안,국토교통위원회,2016-08-11,임기만료폐기,20,http://likms.assembly.go.kr/bill/billDetail.do...,임종성의원 등 11인,http://likms.assembly.go.kr/bill/coactorListPo...,임종성,"강병원,김경협,김병욱,김철민,민홍철,소병훈,안호영,윤후덕,전현희,최경환",자동차관리법,2,2
3,PRC_E1H6R1P1W2U9L1N6D5U7M4W8D6K0G1,2003948,집합건물의 소유 및 관리에 관한 법률 일부개정법률안,법제사법위원회,2016-11-29,임기만료폐기,20,http://likms.assembly.go.kr/bill/billDetail.do...,전현희의원 등 10인,http://likms.assembly.go.kr/bill/coactorListPo...,전현희,"기동민,김영춘,노웅래,박선숙,손혜원,윤호중,윤후덕,임종성,정성호","집합,건물,소유",0,3
4,PRC_P1B7H0I8V3O0D1H4O2O9F2A1I4T6U3,2008863,장기공공임대주택 입주자 삶의 질 향상 지원법 일부개정법률안,국토교통위원회,2017-08-30,수정가결,20,http://likms.assembly.go.kr/bill/billDetail.do...,전현희의원 등 10인,http://likms.assembly.go.kr/bill/coactorListPo...,전현희,"김정우,신창현,안규백,안호영,윤관석,이원욱,임종성,주승용,한정애","장기,임대주택,주자,향상",2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,PRC_C1C6R1U2Q2M6Y1V4N5L9G3K1R7V9D9,2004610,고등교육법 일부개정법률안,교육위원회,2016-12-26,임기만료폐기,20,http://likms.assembly.go.kr/bill/billDetail.do...,서영교의원 등 10인,http://likms.assembly.go.kr/bill/coactorListPo...,서영교,"강창일,김병기,김종대,남인순,박재호,안규백,위성곤,전혜숙,황희","고등,교육법",9,154
3496,PRC_E1Z8A1B2T0B6B2Q0U0M9L4X7A3Y0U7,2017278,광산피해의 방지 및 복구에 관한 법률 일부개정법률안,산업통상자원중소벤처기업위원회,2018-12-06,임기만료폐기,20,http://likms.assembly.go.kr/bill/billDetail.do...,황주홍의원 등 10인,http://likms.assembly.go.kr/bill/coactorListPo...,황주홍,"박주현,이언주,이용주,이종걸,이찬열,장정숙,정동영,정인화,최도자","광산,피해,방지,복구",11,1074
3497,PRC_V1N7G1D2E2P9Q1I4A5G4E4P4Z0U5W1,2011138,소금산업 진흥법 일부개정법률안,농림축산식품해양수산위원회,2017-12-29,대안반영폐기,20,http://likms.assembly.go.kr/bill/billDetail.do...,전혜숙의원 등 12인,http://likms.assembly.go.kr/bill/coactorListPo...,전혜숙,"기동민,김상희,김정우,노웅래,신창현,양승조,이찬열,인재근,장정숙,전해철,정춘숙",소금,1,434
3498,PRC_R1D9C0E9X0U3M1T1H3Y3A5N6P8C1L8,2022311,도로교통법 일부개정법률안,행정안전위원회,2019-09-03,임기만료폐기,20,http://likms.assembly.go.kr/bill/billDetail.do...,백승주의원 등 10인,http://likms.assembly.go.kr/bill/coactorListPo...,백승주,"김무성,박덕흠,박명재,유민봉,장석춘,최교일,최연혜,홍철호,황영철",도로교통법,5,20


In [ ]:
# data save

df_suggest.to_csv('./suggest_20.csv', index=False, encoding='cp949')
df_ppl.to_csv('./people_20.csv', index=False, encoding='cp949')

In [ ]:
# data load

df_suggest = pd.read_csv('./suggest_20.csv', encoding='cp949')
df_ppl= pd.read_csv('./people_20.csv', encoding='cp949')